In [457]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [458]:
def generaPath(tipo, nombre):
    prepath = str(os.getcwd())
    #print(prepath)
    if (tipo == "I"):
        path = prepath + '\\' + nombre
    else :
        path = prepath + '\\generados\\' + nombre
    return path

In [459]:
df_cq_tr = pd.read_csv(generaPath('O', 'df_cq_tr.csv'))
df_cq_re = pd.read_csv(generaPath('O', 'df_cq_re.csv'))

df_cc_tr = pd.read_csv(generaPath('O', 'df_cc_tr.csv'))
df_cc_re = pd.read_csv(generaPath('O', 'df_cc_re.csv'))

df_ct_tr = pd.read_csv(generaPath('O', 'df_ct_tr.csv'))
df_ct_re = pd.read_csv(generaPath('O', 'df_ct_re.csv'))

df_cd_tr = pd.read_csv(generaPath('O', 'df_cd_tr.csv'))
df_cd_re = pd.read_csv(generaPath('O', 'df_cd_re.csv'))

df_cs_tr = pd.read_csv(generaPath('O', 'df_cs_tr.csv'))
df_cs_re = pd.read_csv(generaPath('O', 'df_cs_re.csv'))

df_cs_tr_sb = pd.read_csv(generaPath('O', 'df_cs_tr_sb.csv'))
df_cs_re_sb = pd.read_csv(generaPath('O', 'df_cs_re_sb.csv'))


df_sel_sb_tr = pd.read_csv(generaPath('O','df_sel_sb_tr.csv'))
df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_sb_re.csv'))

df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_si_tr.csv'))
df_sel_si_re = pd.read_csv(generaPath('O','df_sel_si_re.csv') )

sbs = pd.read_csv(generaPath('O','sbs.csv'))
singles = pd.read_csv(generaPath('O','singles.csv'))



In [460]:
def unsplited_tupla(secuence):
    num = str(secuence[0])
    partido = num.split(sep='|') if (num.find('|') >= 1) else num
    return partido

In [461]:
def crea_sorteos(single,sb, secuence):
    sorteo = []
    enteros = []
    #print(type(secuence))
    splitedtupla = unsplited_tupla(secuence)
    print(splitedtupla)
    enteros = [int(et) for et in splitedtupla]
    print("Enteros")
    print(enteros)
    sorteo = singles.iloc[enteros, 0]
    #print(sorteo)
    #sorteo.append(str(sb))
    return sorteo

In [462]:
def secuencelist_builder(a,b,c,d,e):
    secuencia=[]
    secuencia.append(a)
    secuencia.append(b)
    secuencia.append(c) 
    secuencia.append(d)
    secuencia.append(e)
    palabra = '|'.join([str(et) for et in secuencia])
    return palabra


In [463]:
def crea_secuencias():
    secuencias = []
    secuencia = []
    totalconteo = singles.count()[0]
    i = 0
    a = 1
    b = a + 1
    c = b + 1
    d = c + 1
    e = d + 1
    while (a <= totalconteo):
        while(b <= totalconteo):
            while(c <= totalconteo):
                while(d <= totalconteo):
                    while(e <= totalconteo):
                        secuencias.append(secuencelist_builder(a,b,c,d,e))
                        e += 1
                    d += 1
                    e = d + 1
                c += 1
                d = c + 1
                e = d + 1
            b += 1
        a += 1
    return secuencias


In [464]:
def cayo_antes(sbs, singles):
    sorteos = []
    modaa = []
    lasts = []
    delta = []  
    secuencias = []
    #secuencia = [1,2,3,4,5]
    secuencias = crea_secuencias()
    char = ','
    for index, row in sbs.iterrows():
        sb = str(row[0])
        for rows in secuencias:
            #print(type(rows))
            secuencia = str(rows)
            #print(secuencia)
            sorteos.append(crea_sorteos(singles,sb, secuencia))
    dft = sorteos
    #print(dft)
    #proba, proba1,proba2,proba3 = how_hot_is(dft, df_tr_fu_filt_delta)
    #dft['Pr']= proba
    #dft['PrX'] = proba1
    #dft['PrL'] = proba2
    #dft['PrM'] = proba3
    return dft

In [465]:
secus = []
cayo_antes(sbs, singles)
#print(secus)

1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Enteros
[1]
1
Ente

[1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype

In [466]:
cayo_antestetos(sbs, singles)

['1|2|3|4|5', '1|2|3|4|6', '1|2|3|4|7', '1|2|3|4|8', '1|2|3|4|9', '1|2|3|4|10', '1|2|3|4|11', '1|2|3|4|12', '1|2|3|4|13', '1|2|3|4|14', '1|2|3|4|15', '1|2|3|4|16', '1|2|3|5|6', '1|2|3|5|7', '1|2|3|5|8', '1|2|3|5|9', '1|2|3|5|10', '1|2|3|5|11', '1|2|3|5|12', '1|2|3|5|13', '1|2|3|5|14', '1|2|3|5|15', '1|2|3|5|16', '1|2|3|6|7', '1|2|3|6|8', '1|2|3|6|9', '1|2|3|6|10', '1|2|3|6|11', '1|2|3|6|12', '1|2|3|6|13', '1|2|3|6|14', '1|2|3|6|15', '1|2|3|6|16', '1|2|3|7|8', '1|2|3|7|9', '1|2|3|7|10', '1|2|3|7|11', '1|2|3|7|12', '1|2|3|7|13', '1|2|3|7|14', '1|2|3|7|15', '1|2|3|7|16', '1|2|3|8|9', '1|2|3|8|10', '1|2|3|8|11', '1|2|3|8|12', '1|2|3|8|13', '1|2|3|8|14', '1|2|3|8|15', '1|2|3|8|16', '1|2|3|9|10', '1|2|3|9|11', '1|2|3|9|12', '1|2|3|9|13', '1|2|3|9|14', '1|2|3|9|15', '1|2|3|9|16', '1|2|3|10|11', '1|2|3|10|12', '1|2|3|10|13', '1|2|3|10|14', '1|2|3|10|15', '1|2|3|10|16', '1|2|3|11|12', '1|2|3|11|13', '1|2|3|11|14', '1|2|3|11|15', '1|2|3|11|16', '1|2|3|12|13', '1|2|3|12|14', '1|2|3|12|15', '1|2|3

[1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64, 1    25
 Name: tupla, dtype: int64]